# Скрытая марковская модель

В данной работе будет рассмотрена скрытая марковская модель или hidden markov model (HMM) для задачи определения, является ли буква гласной или нет.

## Импорты, загрузка и подготовка данных

In [ ]:
import re

import numpy as np
import nltk
from nltk.tag import hmm
from nltk.corpus import brown
import pandas as pd

In [ ]:
nltk.download('brown')
english = re.compile('^[a-z]+$')

Каждый токен переводим в нижний регистр, оставляем только токены в латинице 

In [ ]:
tokens = []
for sent in brown.sents():
    for w in sent:
        w = w.lower()
        if english.match(w):
                tokens.append(w)
print(f'Number of tokens: {len(tokens)}')

In [ ]:
text = ' '.join(tokens)
len(text), text[:100]

## Unsupervised обучение скрытой марковской модели (Алгоритм Баума-Велша)

Извлечем "словарь" - множество всех букв

In [ ]:
vocab = sorted(list(set(text)))
len(vocab)

Обучение

In [ ]:
trainer = hmm.HiddenMarkovModelTrainer(range(2), vocab)

In [ ]:
tagger = trainer.train_unsupervised([text[:50000]], max_iterations=50)

### Исследуем полученную модель

Матрица переходов $\{a_{ij} = p(s_j|s_i)\}_{i,j = 1}^{|S|}$

In [ ]:
trans_matr = pd.DataFrame(data=np.array([
    [2 ** log_p for log_p in tagger._transitions[0]._data],
    [2 ** log_p for log_p in tagger._transitions[1]._data]
]),
                         columns=[0, 1],
                         index=[0, 1])
trans_matr

In [ ]:
trans_matr.sum(axis=1)

Матрица выходных вероятностей $\{ b_{ij} = p(x_j|s_i) \}_{i, j = 1}^{|S|, |X|}$

In [ ]:
out_matr = pd.DataFrame(data=np.array([
    [2 ** log_p for log_p in tagger._outputs[0]._data],
    [2 ** log_p for log_p in tagger._outputs[1]._data]
]),
                        index=[0, 1],
                        columns=vocab)
out_matr

In [ ]:
out_matr.sum(axis=1)

## Supervised обучение скрытой марковской модели (максимум правдоподобия)

In [ ]:
def make_tag(c):
    if c in 'aeiou':
        return (c,'1')
    else:
        return (c,'0')
supervised = [make_tag(c) for c in text]

In [ ]:
tagger = trainer.train_supervised([supervised[:500]])

### Исследуем полученную модель

Частоты совстречаемостей тегов

In [ ]:
for t in tagger._transitions:
    print(t, tagger._transitions[t].__dict__)

Матрица переходов $\{a_{ij} = p(s_j|s_i)\}_{i,j = 1}^{|S|}$

In [ ]:
trans_matr = pd.DataFrame(data=np.array([
    [tagger._transitions['0'].prob('0'), tagger._transitions['0'].prob('1')],
    [tagger._transitions['1'].prob('0'), tagger._transitions['1'].prob('1')]
]),
                         columns=[0, 1],
                         index=[0, 1])
trans_matr

In [ ]:
trans_matr.sum(axis=1)

Матрица выходных вероятностей $\{ b_{ij} = p(x_j|s_i) \}_{i, j = 1}^{|S|, |X|}$

In [ ]:
out_matr = pd.DataFrame(data=np.array([
    [tagger._outputs['0'].prob(c) for c in vocab],
    [tagger._outputs['1'].prob(c) for c in vocab]
]),
                        index=[0, 1],
                        columns=vocab)
out_matr

In [ ]:
out_matr.sum(axis=1)